# Generate histogram of releases
Code Author: Sahar El Abbadi
Date started: 2023-03-14
Date last edited: 2023-03-14

Notebook objective: to generate a histogram of Stanford releases, categorizing release by type:
- True Positive
- True Negative
- False Negative
- False Positive
- Missing data - overpass recorded in field but not as an overpass by the team
- Stanford Filtered
- Operator Filtered

In [3]:
# Setup
# Imports
from methods_source import load_overpass_summary

In [4]:
# Load operator overpass data

cm_reported = load_overpass_summary(operator='Carbon Mapper', stage=1, strict_discard=False)

# Count true positives
true_positives = cm_reported.query('non_zero_release == True')